# Setup stuff

In [ ]:
!pip install -q transformers diffusers accelerate safetensors --upgrade

In [ ]:
!pip install -U -q autotrain-advanced

In [ ]:
from diffusers import StableDiffusionXLPipeline, DiffusionPipeline, AutoencoderKL
import torch
import itertools
import cv2
import pandas as pd
from random import sample
from PIL import Image
import glob
from google.colab import drive

drive.mount('/content/drive')

Upload data

In [ ]:
!mkdir images
!cp /content/drive/My\ Drive/photobooth/*jpg /content/images/

In [ ]:
class CFG:
    model =  "stabilityai/stable-diffusion-xl-base-1.0"
    data_dir = '/content/images'

In [ ]:
PROJECT_NAME = "DreamboothSDXL"
MODEL_NAME = "stabilityai/stable-diffusion-xl-base-1.0"
REPO_ID = 'konradb/sdxl1'

# Functions

In [ ]:
def image_grid(imgs, rows, cols, resize= 512):
    assert len(imgs) == rows * cols

    if resize is not None:
        imgs = [img.resize((resize, resize)) for img in imgs]

    w, h = imgs[0].size
    grid_w, grid_h = cols * w, rows * h
    grid = Image.new("RGB", size=(grid_w, grid_h))

    for i, img in enumerate(imgs):
        x = i % cols * w
        y = i // cols * h
        grid.paste(img, box=(x, y))

    return grid

# Look around

In [ ]:
imgs = [Image.open(path) for path in glob.glob(CFG.data_dir + "/*.jpg")]
image_grid(imgs, 2, 3)

# Model

In [ ]:
# pipeline setup
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix",
    torch_dtype=torch.float16
)

pipe = StableDiffusionXLPipeline.from_pretrained(
            CFG.model,
             vae=vae,
            torch_dtype=torch.float16,
            variant="fp16",
            use_safetensors=True).to("cuda")


In [ ]:
!autotrain dreambooth --model $MODEL_NAME --project-name $PROJECT_NAME --image-path "/content/images" --prompt "A photo of KPB wearing glasses, looking at the camera, posing for a professional photo"  --resolution 1024 --batch-size 1 --num-steps 500  --gradient-accumulation 4 --lr 1e-4 --mixed-precision "fp16"

In [ ]:
!cp -r DreamboothSDXL/* /content/drive/My\ Drive/photobooth/

In [ ]:
del vae, pipe

In [ ]:

vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix",
    torch_dtype=torch.float16
)
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)

pipe.to("cuda");

In [ ]:
pipe.load_lora_weights('./DreamboothSDXL', weight_name="pytorch_lora_weights.safetensors")

In [ ]:

prompt = "A photo of KPB giving a press conference."

image = pipe(prompt=prompt, num_inference_steps=25, num_images_per_prompt = 4)
image_grid(image.images, 2,2)

In [ ]:
prompt = "A photo of KPB as a private detective in a 1940s film noir "

image = pipe(prompt=prompt, num_inference_steps=25, num_images_per_prompt = 4)
image_grid(image.images, 2, 2)

In [ ]:
prompt = "A photo of KPB as a soldier in a WW2 movie"

image = pipe(prompt=prompt, num_inference_steps=25, num_images_per_prompt = 4)
image_grid(image.images, 2, 2)

In [ ]:
prompt = "A photo of KPB as a populist politician in a rally"

image = pipe(prompt=prompt, num_inference_steps= 30, num_images_per_prompt = 4)
image_grid(image.images, 2, 2)
